In [ ]:
# Authenticate to Earth Engine (run once)
import ee
ee.Authenticate()
ee.Initialize(project='openpas-463512')


In [ ]:
# Interactive exports: precipitation + Sentinel-2 indices (NDVI, NDMI, BSI, MSI)
# - Select variable, year (where applicable), and region(s)
# - Always uses native scale (S2 ~10 m, precipitation 1 km)
# - If a request is too large or fails, falls back to tiled downloads and merges locally (requires rasterio)
# - Outputs GeoTIFFs in `downloads/` (one per region)

import math
import datetime
import pathlib
import shutil
import tempfile
import zipfile
import requests
import ee
import ipywidgets as widgets
from IPython.display import display, clear_output

try:
    import rasterio
    from rasterio.merge import merge as rio_merge
except ImportError:
    rasterio = None

PROJECT = 'openpas-463512'
ASSETS = {
    'aracena': 'projects/openpas-463512/assets/aracena',
    'geres': 'projects/openpas-463512/assets/geres',
    'ordesa': 'projects/openpas-463512/assets/ordesa',
    'quercy': 'projects/openpas-463512/assets/quercy',
    'zorita': 'projects/openpas-463512/assets/zorita',
}
DOWNLOAD_DIR = pathlib.Path('downloads')
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

MONTHLY_BANDS = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']

# ----------------------------- Helpers ------------------------------------

def mask_s2_sr(img: ee.Image) -> ee.Image:
    scl = img.select('SCL')
    mask = (scl.neq(3)
            .And(scl.neq(7))
            .And(scl.neq(8))
            .And(scl.neq(9))
            .And(scl.neq(10))
            .And(scl.neq(11)))
    return img.updateMask(mask)

def scale_sentinel(img: ee.Image) -> ee.Image:
    optical = img.select(['B2','B3','B4','B8','B11','B12']).divide(10000)
    return optical.addBands(img.select('SCL'))

def get_s2_collection(year: int, region: ee.Geometry) -> ee.ImageCollection:
    start = f"{year}-01-01"
    end = f"{year + 1}-01-01"
    col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate(start, end)
           .filterBounds(region)
           .map(mask_s2_sr)
           .map(scale_sentinel))
    return col

def compute_index(index: str, year: int, region: ee.Geometry) -> tuple[ee.Image, int, str]:
    col = get_s2_collection(year, region)
    if index == 'msi':
        col = col.filter(ee.Filter.calendarRange(6, 8, 'month'))
    composite = col.median()
    if index == 'ndvi':
        img = composite.normalizedDifference(['B8', 'B4']).rename('ndvi')
    elif index == 'ndmi':
        img = composite.normalizedDifference(['B8', 'B11']).rename('ndmi')
    elif index == 'bsi':
        swir = composite.select('B11')
        red = composite.select('B4')
        nir = composite.select('B8')
        blue = composite.select('B2')
        num = swir.add(red).subtract(nir.add(blue))
        den = swir.add(red).add(nir).add(blue).add(1e-6)
        img = num.divide(den).rename('bsi')
    elif index == 'msi':
        img = composite.select('B11').divide(composite.select('B8')).rename('msi')
    else:
        raise ValueError(f"Unsupported index: {index}")
    img = img.toFloat()
    try:
        crs = composite.select('B2').projection().crs().getInfo()
    except Exception:
        crs = 'EPSG:4326'
    return img.clip(region), 10, crs

def get_precip(region: ee.Geometry) -> tuple[ee.Image, int, str]:
    dataset = ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01')
    img = dataset.select(MONTHLY_BANDS).reduce(ee.Reducer.sum()).rename('annual_precip_mm').toInt16()
    crs = 'EPSG:4326'
    return img.clip(region), 1000, crs

def download_zip_to_tif(url: str, out_path: pathlib.Path):
    with tempfile.NamedTemporaryFile(suffix='.zip', delete=False) as tmp:
        tmp_path = pathlib.Path(tmp.name)
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    tmp.write(chunk)
    try:
        with zipfile.ZipFile(tmp_path, 'r') as zf:
            tif_members = [n for n in zf.namelist() if n.lower().endswith('.tif')]
            if not tif_members:
                raise RuntimeError('Downloaded zip does not contain a .tif file')
            with zf.open(tif_members[0]) as src, open(out_path, 'wb') as dst:
                shutil.copyfileobj(src, dst, 8192)
    finally:
        if tmp_path.exists():
            tmp_path.unlink()

def download_single(image: ee.Image, region: ee.Geometry, scale: int, crs: str, out_path: pathlib.Path):
    params = {
        'region': region,
        'scale': int(scale),
        'crs': crs,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13,
    }
    url = image.getDownloadURL(params)
    download_zip_to_tif(url, out_path)

def generate_tiles(region: ee.Geometry, scale: int, max_tile_pixels: int = 1500):
    coords = region.bounds().coordinates().getInfo()[0]
    xs = [c[0] for c in coords]
    ys = [c[1] for c in coords]
    minx, maxx = min(xs), max(xs)
    miny, maxy = min(ys), max(ys)
    pixel_deg = scale / 111320.0
    tile_deg = pixel_deg * max_tile_pixels
    nx = max(1, math.ceil((maxx - minx) / tile_deg))
    ny = max(1, math.ceil((maxy - miny) / tile_deg))
    tiles = []
    for i in range(nx):
        for j in range(ny):
            x0 = minx + i * tile_deg
            x1 = minx + (i + 1) * tile_deg if i < nx - 1 else maxx
            y0 = miny + j * tile_deg
            y1 = miny + (j + 1) * tile_deg if j < ny - 1 else maxy
            tile_geom = ee.Geometry.Rectangle([x0, y0, x1, y1], None, False)
            tiles.append(tile_geom)
    return tiles

def download_tiled(image: ee.Image, region: ee.Geometry, scale: int, crs: str, out_path: pathlib.Path):
    if rasterio is None:
        raise RuntimeError('Request too large; install rasterio to enable tiled merge, or reduce region size.')
    tiles = generate_tiles(region, scale)
    tile_paths = []
    for idx, tile in enumerate(tiles):
        tile_path = out_path.with_suffix(f'.tile{idx}.tif')
        url = image.clip(tile).getDownloadURL({
            'region': tile,
            'scale': int(scale),
            'crs': crs,
            'fileFormat': 'GeoTIFF',
            'maxPixels': 1e13,
        })
        print(f"   [TILE] {idx+1}/{len(tiles)} downloading ...")
        download_zip_to_tif(url, tile_path)
        tile_paths.append(tile_path)
    srcs = [rasterio.open(p) for p in tile_paths]
    mosaic, transform = rio_merge(srcs)
    meta = srcs[0].meta.copy()
    meta.update({
        'height': mosaic.shape[1],
        'width': mosaic.shape[2],
        'transform': transform,
    })
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(mosaic)
    for src in srcs:
        src.close()
    for p in tile_paths:
        p.unlink(missing_ok=True)

def download_to_local(image: ee.Image, region: ee.Geometry, scale: int, crs: str, out_path: pathlib.Path):
    try:
        download_single(image, region, scale, crs, out_path)
    except Exception as e:
        if rasterio is None:
            raise
        print('   [WARN] Single download failed; trying tiled download. Error was:', e)
        download_tiled(image, region, scale, crs, out_path)

def build_image(selection: str, year: int, region: ee.Geometry) -> tuple[ee.Image, int, str]:
    if selection == 'precipitation':
        return get_precip(region)
    return compute_index(selection, year, region)

# ----------------------------- UI -----------------------------------------

variable_dropdown = widgets.Dropdown(
    options=[('Precipitation (annual)', 'precipitation'),
             ('NDVI (annual composite)', 'ndvi'),
             ('NDMI (annual composite)', 'ndmi'),
             ('BSI (annual composite)', 'bsi'),
             ('MSI (Jun-Aug)', 'msi')],
    value='ndvi',
    description='Variable:',
    style={'description_width': '130px'}
)

current_year = datetime.datetime.utcnow().year
year_dropdown = widgets.Dropdown(
    options=list(range(2017, current_year + 1)),
    value=current_year,
    description='Year:',
    style={'description_width': '130px'}
)

region_dropdown = widgets.Dropdown(
    options=['All regions'] + list(ASSETS.keys()),
    value='All regions',
    description='Region:',
    style={'description_width': '130px'}
)

run_button = widgets.Button(description='Run export(s)', button_style='success')
out = widgets.Output()

def on_run_clicked(_):
    with out:
        clear_output()
        selection = variable_dropdown.value
        year = year_dropdown.value
        region_choice = region_dropdown.value
        targets = ASSETS.items() if region_choice == 'All regions' else [(region_choice, ASSETS[region_choice])]
        print(f"Variable: {selection} | Year: {year} | Regions: {[k for k,_ in targets]}")
        for name, asset_id in targets:
            print(f"
[INFO] Processing {name} ({asset_id}) ...")
            fc = ee.FeatureCollection(asset_id)
            region = fc.geometry()
            try:
                image, native_scale, crs = build_image(selection, year, region)
                year_tag = '' if selection == 'precipitation' else f"_{year}"
                outfile = DOWNLOAD_DIR / f"{name}_{selection}{year_tag}.tif"
                download_to_local(image, region, native_scale, crs, outfile)
                print(f"[OK] Saved {outfile}")
            except Exception as e:
                print(f"[ERROR] {name}: {e}")
        print('
Done. Check the downloads/ folder.')

run_button.on_click(on_run_clicked)

ui = widgets.VBox([
    variable_dropdown,
    year_dropdown,
    region_dropdown,
    run_button,
    out
])

display(ui)
